# Mount Drive:

In [0]:
#Import drive
from google.colab import drive
#Mount Google Drive%
drive.mount("/content/drive")

In [0]:
import os
os.chdir('drive/My Drive/Work/Thesis_Julien_Dejasmin/Work/code/Binary_activations_V2/MNIST_Binary_V2')
!ls

# Import:

In [0]:
!pip install pytorch-ignite

In [0]:
try:
  from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
  from ignite.metrics import Accuracy, Loss, ConfusionMatrix
  from ignite.handlers import ModelCheckpoint
  from utils.training import run, evaluate
except ImportError:
  raise RuntimeError("no module Ignite, to install Ignite: 'pip install pytorch-ignite'.")

from tqdm import tqdm

In [0]:
import torch
from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision.transforms import Compose, ToTensor, Normalize
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
import numpy as np
from functools import partial

from utils.models import get_my_model_Omniglot, fetch_last_checkpoint_model_filename
from DataLoader.dataLoaders import get_omniglot_dataloaders_classification
from utils.functions import Hardsigmoid

In [0]:
from visualize.viz import visTensor, get_activation, viz_activations, viz_filters
from visualize.viz import viz_heatmap, test_predict_few_examples, standardize_and_clip, format_for_plotting
from visualize.viz import apply_transforms, GradientAscent, get_filter_layer2
from visualize.viz import get_region_layer1, get_region_layer2, get_regions_interest, get_all_regions_max

# for regions extraction
import collections
from functools import partial
import cv2

# Dataset:

In [0]:
batch_size_train = 1000
batch_size_test = 64
# Dataset
train_loader, valid_loader, test_loader = get_omniglot_dataloaders_classification(batch_size_train, batch_size_test)

# Load Models:

In [0]:
# parameters model to load no Binary model
binary = False
maxpooling = False
model_no_binary_stride, name_model_no_binary_stride = get_my_model_Omniglot(binary, maxpooling=maxpooling)

path_model = 'trained_models/Omniglot_classif/No_binary_models/stride/'
if torch.cuda.is_available():
  model_no_binary_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_no_binary_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model_no_binary_stride)

In [0]:
binary = False
maxpooling = True
model_no_binary_maxpooling, name_model_no_binary_maxpooling = get_my_model_Omniglot(binary, maxpooling=maxpooling)

path_model = 'trained_models/Omniglot_classif/No_binary_models/maxpooling/'
if torch.cuda.is_available():
  model_no_binary_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_no_binary_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model_no_binary_maxpooling)

In [0]:
binary = True
maxpooling = False
model_binary_stride, name_model_binary_stride = get_my_model_Omniglot(binary, maxpooling=maxpooling)

path_model = 'trained_models/Omniglot_classif/Binary_models/stride/'
if torch.cuda.is_available():
  model_binary_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_binary_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model_binary_stride)

In [0]:
binary = True
maxpooling = True
model_binary_maxpooling, name_model_binary_maxpooling = get_my_model_Omniglot(binary, maxpooling=maxpooling)

path_model = 'trained_models/Omniglot_classif/Binary_models/maxpooling/'
if torch.cuda.is_available():
  model_binary_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_binary_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model_binary_maxpooling)

In [0]:
binary = False
maxpooling = False
mixt=True
model_mixt_stride, name_model_mixt_stride = get_my_model_Omniglot(binary, maxpooling=maxpooling, mixt=mixt)

path_model = 'trained_models/Omniglot_classif/Mixt_binary_models/stride/'
if torch.cuda.is_available():
  model_mixt_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_mixt_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model_mixt_stride)

In [0]:
binary = False
maxpooling = True
mixt=True
model_mixt_maxpooling, name_model_mixt_maxpooling = get_my_model_Omniglot(binary, maxpooling=maxpooling, mixt=mixt)

path_model = 'trained_models/Omniglot_classif/Mixt_binary_models/maxpooling/'
if torch.cuda.is_available():
  model_mixt_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_mixt_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model_mixt_maxpooling)

## Recap Scores:

### On train dataset:

In [0]:
evaluate(model_no_binary_stride, train_loader)
evaluate(model_no_binary_maxpooling, train_loader)
evaluate(model_binary_stride, train_loader)
evaluate(model_binary_maxpooling, train_loader)
evaluate(model_mixt_stride, train_loader)
evaluate(model_mixt_maxpooling, train_loader)

### On valid dataset:

In [0]:
evaluate(model_no_binary_stride, valid_loader)
evaluate(model_no_binary_maxpooling, valid_loader)
evaluate(model_binary_stride, valid_loader)
evaluate(model_binary_maxpooling, valid_loader)
evaluate(model_mixt_stride, valid_loader)
evaluate(model_mixt_maxpooling, valid_loader)

### On test dataset:

In [0]:
evaluate(model_no_binary_stride, test_loader)
evaluate(model_no_binary_maxpooling, test_loader)
evaluate(model_binary_stride, test_loader)
evaluate(model_binary_maxpooling, test_loader)
evaluate(model_mixt_stride, test_loader)
evaluate(model_mixt_maxpooling, test_loader)

# Preleminary vews:

## Visualization few predictions:

In [0]:
print('No binary model stride')
test_predict_few_examples(model_no_binary_stride, test_loader)
plt.show()
print('No binary model maxpooling')
test_predict_few_examples(model_no_binary_maxpooling, test_loader)
plt.show()
print('Binary model stride')
test_predict_few_examples(model_binary_stride, test_loader)
plt.show()
print('Binary model maxpooling')
test_predict_few_examples(model_binary_maxpooling, test_loader)
plt.show()
print('Mixt model stride')
test_predict_few_examples(model_mixt_stride, test_loader)
plt.show()
print('Mixt model maxpooling')
test_predict_few_examples(model_mixt_maxpooling, test_loader)
plt.show()

## Visualization Activations values for a specific data:

In [0]:
index = 10

In [0]:
viz_activations(model_no_binary_stride, test_loader, index_data)

In [0]:
viz_activations(model_no_binary_maxpooling, test_loader, index_data)

In [0]:
viz_activations(model_binary_stride, test_loader, index_data)

In [0]:
viz_activations(model_binary_maxpooling, test_loader, index_data)

In [0]:
viz_activations(model_mixt_stride, test_loader, index_data)

In [0]:
viz_activations(model_mixt_maxpooling, test_loader, index_data)

## Visualization heatmap for a specific data:

In [0]:
index_data = 10

In [0]:
viz_heatmap(model_no_binary_stride, name_model_no_binary, test_loader, index_data)

In [0]:
viz_heatmap(model_no_binary_maxpooling, name_model_no_binary, test_loader, index_data)

In [0]:
viz_heatmap(model_binary_stride, name_model_no_binary, test_loader, index_data)

In [0]:
viz_heatmap(model_binary_maxpooling, name_model_no_binary, test_loader, index_data)

In [0]:
viz_heatmap(model_mixt_stride, name_model_no_binary, test_loader, index_data)

In [0]:
viz_heatmap(model_mixt_maxpooling, name_model_no_binary, test_loader, index_data)

## Visualization filters trained:

In [0]:
nrow=10

In [0]:
viz_filters(model_no_binary_stride, nrow=nrow)

In [0]:
viz_filters(model_no_binary_maxpooling, nrow=nrow)

In [0]:
viz_filters(model_binary_stride, nrow=nrow)

In [0]:
viz_filters(model_binary_maxpooling, nrow=nrow)

In [0]:
viz_filters(model_mixt_stride, nrow=nrow)

In [0]:
viz_filters(model_mixt_maxpooling, nrow=nrow)

## Visualization generated images for activation maximization with gradient a ascent:

Activation maximization is one form of feature visualization that allows us to visualize what CNN filters are "looking for", by applying each filter to an input image and updating the input image so as to maximize the activation of the filter of interest (i.e. treating it as a gradient ascent task with filter activation values as the loss).

# Visualize Regions that maximized filters:

## Load regions, activations and regions means:

## Visualize regions details:

## Visualize regions resume:

## Histograms:

# ???